In [143]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib .pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [196]:
# Main
i = 1
trials = 1
data = extract_features()
ensamble_list = []
ensamble_list.append(create_classifier(data))
ensemble_error = compute_ensemble_error(data,ensamble_list) # Firts enbamble error
r_size = 10 # Number of instances of training data to generate
c_size = 5 # Number of classifiers in the ensamble
i_max = 10 # Number of iterations
while i < c_size and trials < i_max :
    r = data.append(create_lable_new_examples(data,ensamble_list,r_size))
    ensamble_list.append(create_classifier(r))
    ensemble_error_r = compute_ensemble_error(data,[ensamble_list[-1]])
    if ensemble_error_r < ensemble_error:
        i = i + 1
        ensemble_error = ensemble_error_r 
    else:
        del ensamble_list[-1]
    trials = trials + 1


0    1
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
9    1
dtype: int64
0    1
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
9    1
dtype: int64
0    1
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
9    1
dtype: int64
0    1
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
9    1
dtype: int64
0    1
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
9    1
dtype: int64
0    1
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
9    1
dtype: int64
0    1
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
9    1
dtype: int64
0    1
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
9    1
dtype: int64
0    1
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
9    1
dtype: int64


In [144]:
# Generates dataframe from csv file. factorizes all object attributes ad leaves numeric attributes
def extract_features():
    file_names=['Credit Approval'] # will need to be removed
    for i in file_names:
        file_directory = 'Datasets/'+i+'.csv'
        data = pd.read_csv(file_directory)
        data = data.replace(['?',' '], np.nan) # replace all empty values to np.nan
        data = data.dropna(axis=0,how='any') # remove all rows with any np.nan
    for i in data.columns:
        if data[i].dtype == object:
            data[i] = pd.factorize(data[i])[0]
    return data

In [195]:
# Recieves dataframe and outputs the model - In this case, decision tree
def create_classifier(data):
    x = data.iloc[:, :-1]
    y = data.iloc[:, -1]
    x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.33,random_state=101)
    dtree = DecisionTreeClassifier()
    dtree.fit(x_train,y_train)
    #import pdb; pdb.set_trace()
    return dtree

In [163]:
# Recieves data set and list of models. returns the confusion matrix of the majority vote of the ensamble
def compute_ensemble_error(data,ensamble_list):
    x = data.iloc[:, :-1]
    y = data.iloc[:, -1]
    x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.33,random_state=101)
    #import pdb; pdb.set_trace()
    comulative_error = 0 
    predictions_list = []
    for dtree in ensamble_list: # generate list if all classifiers classifications
        predictions_list.append(dtree.predict(x_test))
    c_star =pd.DataFrame(pd.DataFrame(predictions_list).T.apply(lambda x: x.value_counts(), axis = 1).idxmax(axis=1))
    #import pdb; pdb.set_trace()
    ensamble_error = 1 - accuracy_score(y_test,c_star)
    return (ensamble_error)

In [184]:
# Recieves data frame (DF) and r_size, returns r_size DF of randomly generated values with the structure of the DF
def create_training_examples(data,r_size):
    mean = data.mean()
    std = data.std()
    dic = dict()
    for i,col in enumerate(data.columns):
        dic[col] = np.random.normal(mean[i],std[i],r_size)
    d = pd.DataFrame(dic)
    return d

In [194]:
# Recieves dataframe (DF), list of models (ensable) and r_size. Returns DF with r_size new exmples with lables that are opposite
# to the majority vote of the ensamle (see idxmin())
def create_lable_new_examples(data,ensamble_list,r_size):
    r_examples = create_training_examples(data,r_size)
    predictions = pd.DataFrame([[0]*r_size,[1]*r_size]).T
    #import pdb; pdb.set_trace()
    for i,model in enumerate(ensamble_list):
        predictions.append(pd.DataFrame(ensamble_list[i].predict(r_examples.iloc[:,:-1])),ignore_index = True)        
    predictions = predictions.apply(lambda x: x.value_counts(), axis = 1).idxmin(axis=1)
    print(predictions)
    r_examples[data.columns[-1]] = predictions

    return (r_examples)